In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#data01 = pd.read_csv('1429_1.csv', on_bad_lines='skip', engine='python')
#data02 = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv', on_bad_lines='skip', engine='python')
#data03 = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv', on_bad_lines='skip', engine='python')

#data1 = data01 [['name', 'categories']]
#data2 = data02[['name', 'categories']]
#data3 = data03[['name', 'categories']]

#comb_data = pd.concat([data1, data2, data3], ignore_index=True)

# Remove duplicate rows
#comb_data = comb_data.drop_duplicates()

In [5]:
comb_data = pd.read_csv('/content/comb_data.csv')
comb_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67992 entries, 0 to 67991
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              67992 non-null  object 
 1   name            61232 non-null  object 
 2   brand           67992 non-null  object 
 3   reviews.date    67953 non-null  object 
 4   categories      67992 non-null  object 
 5   reviews.rating  67959 non-null  float64
 6   reviews.text    67991 non-null  object 
 7   reviews.title   67973 non-null  object 
dtypes: float64(1), object(7)
memory usage: 4.2+ MB


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
import pandas as pd # Ensure pandas is imported if not already

# The 'comb_data' variable from the previous cell is a DataFrame.
# The operations involving '.str' accessor are intended for a Pandas Series (e.g., a single column).
# We need to explicitly select the 'categories' column for string operations.

# Filter the original 'comb_data' DataFrame based on the length of the 'categories' column.
# Use .copy() to avoid SettingWithCopyWarning later if we modify filtered_comb_data.
filtered_comb_data = comb_data[comb_data['categories'].str.len() > 10].copy()

# Now, prepare the 'categories' column from the filtered DataFrame for TF-IDF vectorization.
text_data_for_vectorization = (
    filtered_comb_data['categories'] # Select the 'categories' Series
    .fillna('')
    .str.lower()
    .str.replace(r'[^a-z ]', '', regex=True)
)

custom_stopwords = ['amazon','new', 'pack', 'th','set','Fire','allnew', 'Kids','special','offer','gb', 'Edition', 'wifi', 'hd', 'usb', 'display'] # Define custom_stopwords here

vectorizer = TfidfVectorizer(
    max_df=0.7,
    min_df=5,
    stop_words=list(ENGLISH_STOP_WORDS) + custom_stopwords,
    ngram_range=(1,2)
)

X = vectorizer.fit_transform(text_data_for_vectorization)

from sklearn.cluster import KMeans

k = 6
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(X)

# Create a DataFrame for results, starting with the filtered data.
comb_data_df = filtered_comb_data # This now contains 'name' and 'categories' from filtered data
comb_data_df["cluster"] = clusters # Add 'cluster' as a new column

terms = vectorizer.get_feature_names_out()

for i in range(k):
    center_terms = kmeans.cluster_centers_[i]
    top_terms = np.argsort(center_terms)[-10:]
    print(f"\nCluster {i}:")
    print([terms[t] for t in top_terms])

cluster_map = {

    0: "Ebook Readers & Tablets",
    1: "Power & Batteries",
    2: "Audio & Entertainment Devices",
    3: "Non-Electronics",
    4: "Accessories"
}

# Now, comb_data_df["cluster"] is a Pandas Series within a DataFrame, and .map() will work
comb_data_df["meta_category"] = comb_data_df["cluster"].map(cluster_map)
comb_data_df.tail(100)

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['edition', 'kids'] not in stop_words.
  warnings.warn(



Cluster 0:
['familyfire tv', 'devicesfeatured brandsamazon', 'devicesfeatured', 'brandsamazon devicesholiday', 'devicesholiday', 'audio', 'media', 'home theaterstreaming', 'theaterstreaming', 'home']

Cluster 1:
['suppliesbatteries chargershealth', 'suppliesbatteries', 'personal care', 'photobatterieshousehold', 'household', 'householdcamcorder batteriescamera', 'carehealth personal', 'carehealth', 'care', 'householdcamcorder']

Cluster 2:
['tabletstabletscomputers tabletsall', 'tabletsall tabletselectronics', 'tabletsfrys', 'movies musicelectronicsipad', 'musicelectronicsipad tabletsandroid', 'musicelectronicsipad', 'tabletsandroid tabletsfrys', 'tabletsall', 'tabletselectronics tech', 'tabletstabletscomputers']

Cluster 3:
['tabletselectronicskids tabletsipad', 'tabletselectronics featurescomputer', 'tabletselectronicskids', 'tabletsipad tabletscases', 'bagselectronics tech', 'bagselectronics', 'accessoriestabletstop', 'accessoriestabletstop ratedamazon', 'computersfire tabletselect

,id,name,brand,reviews.date,categories,reviews.rating,reviews.text,reviews.title,cluster,meta_category
67892,AVqkIhxunnc1JgDc3kg_,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",Amazon,2017-03-31T00:00:00.000Z,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",5.0,Amazon Fire HD8 is one of the best deal at 89$...,Very good tablet and best price,4,Accessories
67893,AVqkIhxunnc1JgDc3kg_,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",Amazon,2017-04-14T00:00:00.000Z,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",4.0,I bought the Fire HD 8 & CaseI use it mostly f...,Very Good Tablet for the Money,4,Accessories
67894,AVqkIhxunnc1JgDc3kg_,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",Amazon,2017-01-01T00:00:00.000Z,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",5.0,"OK its not an iPad, but it is a very good tabl...",Very Good!,4,Accessories
67895,AVqkIhxunnc1JgDc3kg_,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",Amazon,2017-01-31T00:00:00.000Z,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",5.0,this tablet is awesome. the fact that I can us...,very happy,4,Accessories
67896,AVqkIhxunnc1JgDc3kg_,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",Amazon,2017-03-27T00:00:00.000Z,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",5.0,"Very happy with this purchase, it was exactly ...",Very Happy,4,Accessories
...,...,...,...,...,...,...,...,...,...,...
67987,AVqkIhxunnc1JgDc3kg_,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",Amazon,2016-12-07T00:00:00.000Z,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",5.0,I got 2 of these for my 8 yr old twins. My 11 ...,Xmas gift,4,Accessories
67988,AVqkIhxunnc1JgDc3kg_,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",Amazon,2017-01-20T00:00:00.000Z,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",4.0,I bought this for my niece for a Christmas gif...,yes it is a great tablet.,4,Accessories
67989,AVqkIhxunnc1JgDc3kg_,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",Amazon,2017-01-30T00:00:00.000Z,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",5.0,"Very nice for light internet browsing, keeping...",You get a lot for the price!,4,Accessories
67990,AVqkIhxunnc1JgDc3kg_,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",Amazon,2017-02-17T00:00:00.000Z,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",5.0,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,4,Accessories
